# Complete dataset (active + bankrupt) estimators

Librerie varie da installare

In [ ]:
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn

Inclusione delle librerie utilizzate

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Variabili di gestione files

In [ ]:
# Path of the files, can be changed
PATH_ACTIVE_DATASET = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset_filtrato\indexes_active.pkl"

PATH_BANKRUPT_DATASET = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset_filtrato\indexes_bankruptcy.pkl"

# True = export summary file in the OUTPUT_PATH
to_export = False

# Path of the desired output file, can be changed
OUTPUT_PATH = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset_filtrato"

Leggo il dataset composto dai 2 file pickle

In [ ]:
active_dataset = pd.read_pickle(PATH_ACTIVE_DATASET)
bankrupt_dataset = pd.read_pickle(PATH_BANKRUPT_DATASET)

Unisco i 2 dataset

In [ ]:
complete_dataset = pd.concat([active_dataset, bankrupt_dataset])

Analizzo il dataset completo

In [ ]:
pd.set_option('display.float_format', lambda x: '%0.4f' % x)
complete_dataset.describe().T

Prelevo solo i campi degli indici finanziari

In [ ]:
indexes_dataset = complete_dataset.iloc[: , 41:].copy()
indexes_corr = indexes_dataset.corr()

In [ ]:
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(indexes_corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(indexes_corr, mask=mask, cmap=cmap, vmax=1,
            center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
indexes_dataset.hist(figsize=(15, 15))

Esporto il dataset completo

In [ ]:
str_cols = ['Ragione sociale',
            'Province',
            'Legal Form',
            'Legal Status',
            'Accounting closing date']
export_dataset = indexes_dataset.copy()
export_dataset[str_cols] = complete_dataset[str_cols]

In [ ]:
# Reorder columns
export_dataset = export_dataset[['Ragione sociale',
                                 'Province',
                                 'Accounting closing date',
                                 'Legal Form',
                                 'Legal Status',
                                 'PN/Totale Debiti',
                                 'Deb. Prev + Trib/Attivo',
                                 'Tempo medio riscossione (TMR)',
                                 'Tempo medio di pagamento (TMP)',
                                 'PFN/EBITDA',
                                 'PFN/PN',
                                 'Gearing',
                                 'ROS',
                                 'Working capital/net sales',
                                 'Cash/Current Liabilities',
                                 'Accounts receivable/inventory',
                                 'EBIT/interest expenses',
                                 'Att.Br/Attivo',
                                 'Ricavi/Attivo',
                                 'EBITDA/Totale Debiti']]

In [ ]:
export_dataset.describe().T

Esporto in csv e pickle

In [ ]:
if to_export:
    export_dataset.to_csv(OUTPUT_PATH + "/indexes_complete.csv")
    export_dataset.to_pickle(OUTPUT_PATH + "/indexes_complete.pkl")

Cerco outliers

In [ ]:
indexes_cols = [ 'PN/Totale Debiti',
                 'Deb. Prev + Trib/Attivo',
                 'Tempo medio riscossione (TMR)',
                 'Tempo medio di pagamento (TMP)',
                 'PFN/EBITDA',
                 'PFN/PN',
                 'Gearing',
                 'ROS',
                 'Working capital/net sales',
                 'Cash/Current Liabilities',
                 'Accounts receivable/inventory',
                 'EBIT/interest expenses',
                 'Att.Br/Attivo',
                 'Ricavi/Attivo',
                 'EBITDA/Totale Debiti']
for index_col in indexes_cols:
    plt.figure()
    sns.boxplot(x=export_dataset[index_col])

Rimuovo outliers

In [ ]:
filtered_export_dataset = pd.DataFrame()
for index_col in indexes_cols:
    Q1 = export_dataset[index_col].quantile(0.25)
    Q3 = export_dataset[index_col].quantile(0.75)
    IQR = Q3 - Q1
    MIN = max(export_dataset[index_col].min(), Q1 - 1.5 * IQR)
    MAX = min(export_dataset[index_col].max(), Q1 + 1.5 * IQR)

    filtered_export_dataset = export_dataset[(export_dataset[index_col] >= MIN) & (export_dataset[index_col] <= MAX)]
    plt.figure()
    sns.boxplot(x=filtered_export_dataset[index_col])

Analizzo il dataset con outlier rimossi

In [ ]:
filtered_export_dataset.describe().T

In [ ]:
indexes_corr = filtered_export_dataset.corr()
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(indexes_corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(indexes_corr, mask=mask, cmap=cmap, vmax=1,
            center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Esporto in csv e pickle

In [ ]:
if to_export:
    filtered_export_dataset.to_csv(OUTPUT_PATH + "/filtered_indexes_complete.csv")
    filtered_export_dataset.to_pickle(OUTPUT_PATH + "/filtered_indexes_complete.pkl")